In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta
import time
import requests
import os
import import_ipynb
from threading import Timer
import abc
from ordered_set import OrderedSet
import configparser
from twilio.rest import Client
import mplfinance as mpf
import backtrader as bt
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.set_option('display.max_rows', None)
pd.set_option('mode.chained_assignment', 'raise')
                                                  
"""
TODO:
- Backtester
- Separate into different files w proper imports

"""

'\nTODO:\n- Backtester\n- Separate into different files w proper imports\n\n'

In [2]:
class StockAlert:
    def __init__(self, ticker, signal, signal_date, close_price):
        self.ticker = ticker
        self.signal = signal
        self.date = signal_date
        self.price = close_price
        self.strategy = ""
        
        if signal == 1:
            self.strategy = "BUY"
        elif signal == -1:
            self.strategy = "SELL"
        
    def __str__(self):
        return "ALERT [%s]: %s %s at %s" % (self.date, self.strategy, self.ticker, self.price)

class PortfolioAlerter:
    def __init__(self, tickers : [str]):
        self.tickers = tickers
        self.data = TDDataFetcher(tickers).get_daily_price_history(datetime(2020, 1, 1))
        self.data["Signal"] = 0
        self.client = Client("AC069dc7540b34e4d53552123ac96f97ff", "5b460db1d5b41a9436bec81aaaa6d515")
        
        EMA20 = []
        EMA50 = []

        count = 0

        # Assuming we have same # of data points for every stock !!
        for ticker in OrderedSet(self.data.index.get_level_values(0)):
            close = self.data.loc[ticker, 'Close']
            current_EMA20 = close.ewm(span=20,adjust=False).mean()
            current_EMA50 = close.ewm(span=50,adjust=False).mean()
            EMA20.extend(current_EMA20)
            EMA50.extend(current_EMA50)


            for i in range(1,len(close)):
                if current_EMA50[i] > current_EMA20[i] and current_EMA50[i-1] < current_EMA20[i-1]:
                    if close[i] < current_EMA20[i] and close[i] < current_EMA50[i]:
        #                 data.loc[ticker].iloc[i, data.columns.get_loc("Signal")] = -1 # SELL
                        self.data.iloc[i + count * int(len(self.data)/len(tickers)), self.data.columns.get_loc("Signal")] = -1 # SELL
                elif current_EMA50[i] < current_EMA20[i] and current_EMA50[i-1] > current_EMA20[i-1]:
                    if close[i] > current_EMA20[i] and close[i] > current_EMA50[i]:
        #                 self.data.loc[ticker].iloc[i, self.data.columns.get_loc("Signal")] = 1 # BUY
                        self.data.iloc[i + count * int(len(self.data)/len(tickers)), self.data.columns.get_loc("Signal")] = 1 # BUY

            count += 1
        
        self.data["20EMA"] = EMA20
        self.data["50EMA"] = EMA50
        
#         self._computeAlerts()
        
    def computeAlerts(self):
        alerts = []
        
        for ticker in OrderedSet(self.data.index.get_level_values(0)):
            tail = self.data.loc[ticker].tail(3)
            tail = tail.loc[tail["Signal"] != 0]
            
            if tail["Signal"].any():
                row_data = tail.iloc[-1]
                alert = StockAlert(ticker, row_data["Signal"], row_data.name, row_data["Close"])
                alerts.append(str(alert))
        
        if len(alerts) > 0:
            self._sendAlerts(alerts)
                
    def _sendAlerts(self, alerts):
        alert_msg = '\n'.join(alerts)
        print(alert_msg)
        self.client.messages.create(to="+19149800095", from_="+14159388441", body=alert_msg)
        
# Methods to run the alerter:
        
# Run at a future date
def main_alert():
        SP500_TICKERS = pd.read_csv("SP500.csv")['Symbol'].tolist()[20:50]
        pa = PortfolioAlerter(SP500_TICKERS)
        pa.computeAlerts()
            
def run_at_future_date():
    while True:
        now = datetime.today()
        later = now.replace(day=now.day, hour= 2, minute=0, second=0, microsecond=0) #+ timedelta(days=1)
        delta = later - now
        secs = delta.total_seconds()

        t = Timer(secs, main_alert)
        t.start()
        
# Run infinitely every X amount of time
def run_every_time_interval():
    sleep_hours = (12) * 60 * 60

    while 1: 
        main_alert()
        dt = datetime.now() + timedelta(hours=12) 
        dt = dt.replace(minute=10)

        while datetime.now() < dt:
            time.sleep(sleep_hours)
    

In [3]:
class DataGenerator(metaclass=abc.ABCMeta):
    """
        Abstract base class for aquiring stock data
    """
    def __init__(self, tickers : [str]):
        self.tickers = tickers
        self._data = None
    
    def get_tickers(self):
        return self.tickers
        
    def has_dataframe(self):
        if self._data is None:
            return True
        return False
    
    def get_dataframe(self):
        return self._data
    
    @abc.abstractmethod
    def get_data_for(self, ticker : str):
        pass

class DataFetcher(DataGenerator):
    """
        Abstract base class for fetching historical stock data from external API's and websites
    """
    def __init__(self, tickers : [str]):
        super(DataFetcher, self).__init__(tickers)
        
    @abc.abstractmethod
    def _get_price_history(self, start_date : datetime, end_date : datetime):
        pass
        
    @abc.abstractmethod
    def get_daily_price_history(self, start_date : datetime, end_date=date.today()):
        """
        params:
            - start_date : datetime object
            - end_date : datetime object
        return:
            - dataframe of daily price history
        """
        pass
    
class TDDataFetcher(DataFetcher):
    def __init__(self, tickers : [str], config_path = ""):
        super(TDDataFetcher, self).__init__(tickers)
        config = configparser.RawConfigParser()
        if config_path is "":
            home_dir = os.path.dirname(os.path.dirname(os.getcwd()))
            config.read('%s/config/config.ini' % home_dir)
        else:
            config.read(config_path)
        self.__apiKey = config.get('TD', 'apiKey') # TODO : tthrow error if no key found
      
    def _get_price_history(self, start_date : datetime, end_date : datetime, params : dict):
        data = []
        count = 0
        for ticker in self.tickers:
            url = "https://api.tdameritrade.com/v1/marketdata/%s/pricehistory" % ticker
            out = requests.get(url=url,params=params).json()
#             print(out)
            if out['empty']:
                # TODO : throw error
#                 print("FALSE")
                self.tickers.remove(ticker)
                continue
            df = pd.DataFrame(out['candles']).rename(columns={'datetime':'Date','close':'Close', 'high':'High', 'low':'Low','open':'Open','volume':'Volume'}).set_index('Date')
            df.index = [datetime.fromtimestamp(i/1000).date() for i in df.index]
            df = df[['High','Low','Open','Close','Volume']]
            df.index = pd.to_datetime(df.index)
            data.append(df)
            count += 1
            if count % 110 == 0: # stay below transactions/second limit
                time.sleep(30)
        datas = map (data, self.tickers)
        self._df = pd.concat(data, keys=self.tickers, names=['Ticker', 'Date'])
        return self._df   
    
    def get_quotes(self):
        params = {'apikey':self.__apiKey, 'symbol':','.join(self.tickers)}
        url = "https://api.tdameritrade.com/v1/marketdata/quotes"
        out = requests.get(url=url,params=params).json()
        df = pd.DataFrame(out)
        if df.empty:
            print("FALSE")
            return None
        return df.transpose()
    
    def get_daily_price_history(self, start_date : datetime, end_date=date.today()):
        params = {'apikey':self.__apiKey, 'startDate':int(start_date.strftime('%s'))*1000, 'endDate':int(end_date.strftime('%s'))*1000, 'periodType':'month', 'frequencyType':'daily'}
        return self._get_price_history(start_date, end_date, params)
    
    def get_data_for(self, ticker : str):
        if self.has_dataframe():
            print("[ERROR] [TDDataFetcher] fetching dataframe : never fetched pricing data. Ensure to call get_daily_price_history first.")
            return None
        return self._df.loc[ticker]
    
class Chartable(metaclass=abc.ABCMeta):
    def __init__(self, data, style="yahoo"):
        self._data = data
        self.date_str = datetime.strptime(str(data.index[0][1]), '%Y-%m-%d %H:%M:%S').strftime('%m/%d/%y') + ' - ' + datetime.strptime(str(data.index[-1][1]), '%Y-%m-%d %H:%M:%S').strftime('%m/%d/%y')
        self.__mav_colors = mavcolors=['Brown','Orange', 'Green', 'Blue', 'Magenta', 'Brown', 'Grey']
        self.__style = mpf.make_mpf_style(base_mpf_style=style, y_on_right=False, edgecolor='Black', facecolor='White', mavcolors=self.__mav_colors)
          
    @abc.abstractproperty
    def __name__(self):
        pass
        
    def _plot(self, data, name, type_name='candle', desc='', apds=[], mav=[], hlines=[], add_plots=[]):
#         add_plots = [mpf.make_addplot(self.__add_mav(avg),linestyle='solid') for avg in mav]
#         apds.append(add_plots)
        fig, axs = mpf.plot(data, addplot=add_plots, figscale=1, figratio=(2*8,2*5.75), type=type_name, volume=True, hlines=hlines, style=self.__style, title='\n\n%s\n%s' % (name, self.date_str), returnfig=True)
        
        for avg in mav:
            axs[0].plot(np.arange(len(data.index)), self.__add_mav(avg))
            
        return fig, axs
    
    def plot_data_for(self, ticker, type_name='candle', mavs=[], signal=False):
        """
        ticker : str of ticker name to query
        type_name : candle, ohlc, line
        mavs : column names in the dataframe
        """
        add_plots = []
        data = self._data.loc[ticker]
        
        if len(mavs) > 0:
            add_plots = [mpf.make_addplot(data[mav],linestyle='solid') for mav in mavs]
            
        if signal:
            signal_high = [np.nan]*len(data)
            signal_low = [np.nan]*len(data)
            count = 0
            
            for index, row in data.iterrows():
                if row["Signal"] < 0:
                    signal_low[count] = row["Close"] * 1.05
                elif row["Signal"] > 0:
                    signal_high[count] = row["Close"] * 0.95
                    
                count += 1
                
            add_plots.extend([mpf.make_addplot(signal_low,type='scatter',markersize=200,marker='v',color='r'),mpf.make_addplot(signal_high,type='scatter',markersize=200,marker='^',color='g')])
                    
        fig, axs = self._plot(data, ticker, type_name=type_name, add_plots=add_plots)
        return fig, axs
        
  

In [4]:
# class PandasData(bt.feed.DataBase):
#     '''
#     The ``dataname`` parameter inherited from ``feed.DataBase`` is the pandas
#     DataFrame
#     '''

#     params = (
#         # Possible values for datetime (must always be present)
#         #  None : datetime is the "index" in the Pandas Dataframe
#         #  -1 : autodetect position or case-wise equal name
#         #  >= 0 : numeric index to the colum in the pandas dataframe
#         #  string : column name (as index) in the pandas dataframe
#         ('datetime', None),

#         # Possible values below:
#         #  None : column not present
#         #  -1 : autodetect position or case-wise equal name
#         #  >= 0 : numeric index to the colum in the pandas dataframe
#         #  string : column name (as index) in the pandas dataframe
#         ('open', 'Open'),
#         ('high', 'High'),
#         ('low', 'Low'),
#         ('close', 'Close'),
#         ('volume', 'Volumne'),
#         ('openinterest', None),
#     )
    
class EMACrossoverStrategy(bt.Strategy):
    params = (('pfast',20),('pslow',50),)
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.order = None
        self.fast_ema = bt.indicators.EMA(self.dataclose, period=self.params.pfast)
        self.slow_ema = bt.indicators.EMA(self.dataclose, period=self.params.pslow)

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
        
        if self.position:
            if self.slow_ema[0] > self.fast_ema[0] and self.slow_ema[-1] < self.fast_ema[-1]:
                if self.dataclose[0] < self.fast_ema[0] and self.dataclose[0] < self.slow_ema[0]:
                    self.log('SELL CREATE, %.2f' % self.dataclose[0])
                    self.order = self.sell()
        else:
            if self.slow_ema[0] < self.fast_ema[0] and self.slow_ema[-1] > self.fast_ema[-1]:
                if self.dataclose[0] > self.fast_ema[0] and self.dataclose[0] > self.slow_ema[0]:
                    self.log('BUY CREATE, %.2f' % self.dataclose[0])
                    self.order = self.buy()
#         else:
            # We are already in the market, look for a signal to CLOSE trades
#             if len(self) >= (self.bar_executed + 5):
#                 self.log(f'CLOSE CREATE {self.dataclose[0]:2f}')
#                 self.order = self.close()
                
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # An active Buy/Sell order has been submitted/accepted - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, {order.executed.price:.2f}')
            elif order.issell():
                self.log(f'SELL EXECUTED, {order.executed.price:.2f}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Reset orders
        self.order = None

In [4]:
# class Strategy(metaclass=abc.ABCMeta):
    
#     def __init__(self):
#         pass
    
#     @abc.abstractmethod
#     def execute(self):
#         pass
    
# class FastSlowEMAStrategy(Strategy):
    
#     def __init__(self, fast_ema, slow_ema):
#         super(FastSlowEMAStrategy, self).__init__()
#         self.fast_ema = fast_ema
#         self.slow_ema = slow_ema
        
#     def execute(self, data, tickers):
        
#         data["Signal"] = 0
#         count = 0
        
#         for ticker in OrderedSet(data.index.get_level_values(0)):
#             close = data.loc[ticker, 'Close']
            
#             for i in range(1,len(close)):
#                 current_fast = data.loc[ticker, self.fast_ema][i]
#                 current_slow = data.loc[ticker, self.slow_ema][i]
                
#                 if current_slow > current_fast and data.loc[ticker, self.slow_ema][i-1] < data.loc[ticker, self.fast_ema][i-1]:
#                     if close[i] < current_fast and close[i] < current_slow:
#                         data.iloc[i + count * int(len(data)/len(tickers)), data.columns.get_loc("Signal")] = -1 # SELL
#                 elif current_slow < current_fast and data.loc[ticker, self.slow_ema][i-1] > data.loc[ticker, self.fast_ema][i-1]:
#                     if close[i] > current_fast and close[i] > current_slow:
#                         data.iloc[i + count * int(len(data)/len(tickers)), data.columns.get_loc("Signal")] = 1 # BUY

#             count += 1
            
#         return data
        
class StockDataManager(Chartable):
    """
    For Interday trading
    
    Caution: Assumes we have the same amount of data points for all given tickers
    """
    
    def __init__(self, data_fetcher : DataFetcher, start_date : datetime): # TODO : generalized to DataLoader then DataGenerator
        super(StockDataManager, self).__init__(data_fetcher.get_daily_price_history(start_date))
        self.tickers = data_fetcher.get_tickers()
#         self._data = data_fetcher.get_daily_price_history(start_date)
        
    def get_dataframe(self):
        return self._data
    
    def get_data_for(self, ticker : str):
        try:
            return self._data.loc[ticker]
        except KeyError:
            print(f"{ticker} was not passed to the StockDataManager upon construction")
            return pd.DataFrame()
    
#     def add_EMAs(self, emas : [int]):

#         ema_list = np.empty((len(emas),len(sm._data)))
#         ticker_data_len = int(len(sm._data) / len(tickers))
#         count = 0

#         for ticker in OrderedSet(self._data.index.get_level_values(0)):
#             close = self._data.loc[ticker, 'Close']
            
#             for i in range(len(emas)):
#                 ema_list[i][ticker_data_len * count : ticker_data_len * (count+1)] = close.ewm(span=emas[i],adjust=False).mean()
            
#             count += 1
        
#         for i in range(len(emas)):  
#             self._data["%dEMA" % emas[i]] = ema_list[i]
            
    
#     def add_SMAs(self, smas : [int]):

#         sma_list = np.empty((len(smas),len(sm._data)))
#         ticker_data_len = int(len(sm._data) / len(tickers))
#         count = 0

#         for ticker in OrderedSet(self._data.index.get_level_values(0)):
#             close = self._data.loc[ticker, 'Close']
            
#             for i in range(len(smas)):
#                 sma_list[i][ticker_data_len * count : ticker_data_len * (count+1)] = close.rolling(window=smas[i]).mean()#.ewm(span=emas[i],adjust=False).mean()
            
#             count += 1
        
#         for i in range(len(smas)):  
#             self._data["%dSMA" % smas[i]] = sma_list[i]
            
    def __name__(self):
        return ''.join(self.tickers)
    
    def run_strategy(self, strategy : bt.Strategy):
         # Create a cerebro entity
        cerebro = bt.Cerebro()

        # Add a strategy
        cerebro.addstrategy(EMACrossoverStrategy)
        
        #Loop through the tickers adding data to cerebro.
        for ticker in self.tickers:
            data = bt.feeds.PandasData(dataname=self._data.loc[ticker], timeframe=bt.TimeFrame.Days, openinterest=None)
            cerebro.adddata(data, name=ticker)        

        # Set our desired cash start
        cerebro.broker.setcash(100000.0)

        start_cash = cerebro.broker.getvalue()

        # Print out the starting conditions
        print('Starting Portfolio Value: %.2f' % start_cash)

        # Run over everything
        cerebro.run()

        end_cash = cerebro.broker.getvalue()

        # Print out the final result
        print('Final Portfolio Value: %.2f' % end_cash)
        print('PnL Value: %.2f' % (end_cash - start_cash))

        cerebro.plot(style='candlestick')
    
    def run_strategy_for(self, ticker, strategy : bt.Strategy):
        # Create a cerebro entity
        cerebro = bt.Cerebro(stdstats=False)

        # Add a strategy
        cerebro.addstrategy(EMACrossoverStrategy)

        data = bt.feeds.PandasData(dataname=self._data.loc[ticker], timeframe=bt.TimeFrame.Days, openinterest=None)

        # Add the Data Feed to Cerebro
        cerebro.adddata(data, name=ticker)

        # Set our desired cash start
        cerebro.broker.setcash(100000.0)

        start_cash = cerebro.broker.getvalue()

        # Print out the starting conditions
        print('Starting Portfolio Value: %.2f' % start_cash)

        # Run over everything
        cerebro.run()

        end_cash = cerebro.broker.getvalue()

        # Print out the final result
        print('Final Portfolio Value: %.2f' % end_cash)
        print('PnL Value: %.2f' % (end_cash - start_cash))

        cerebro.plot()
            
#     def add_strategy(self, strategy : Strategy):
#         strategy.execute(self._data, self.tickers)
    
    def chart_for(self, ticker : str, type_name='candle', mavs=[], signal=False):
        return self.plot_data_for(ticker, type_name, mavs, signal)
    
#     def chart(self, tickers : [str]):
#         data = self._data.loc[tickers]
# #         self.plot_candlesticks()
#         pass

NameError: name 'Chartable' is not defined

In [3]:
tickers = ["TDOC", "PYPL"]
sm = StockDataManager(TDDataFetcher(tickers), datetime(2019, 6, 1))
# sm.add_EMAs([20,50])
# sm.add_strategy(FastSlowEMAStrategy("20EMA", "50EMA"))
# sm.run_strategy_for("TDOC", EMACrossoverStrategy)
sm.run_strategy(EMACrossoverStrategy)
# sm.chart_for("TDOC", type_name='line')#, mavs=["20EMA","50EMA"], signal=True)
# sm.test_strategy()

NameError: name 'StockDataManager' is not defined

In [ ]:
def runstrat(dataframe):
    # Create a cerebro entity
    cerebro = bt.Cerebro(stdstats=False)

    # Add a strategy
    cerebro.addstrategy(EMACrossoverStrategy)

#     dataframe = sm._data.loc[ticker]#pd.read_pickle("data/"+_Symbol+""+cTF+".pkl")
    data = bt.feeds.PandasData(dataname=dataframe, timeframe=bt.TimeFrame.Days, openinterest=None)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data, name=)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    
    start_cash = cerebro.broker.getvalue()

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % start_cash)

    # Run over everything
    cerebro.run()
    
    end_cash = cerebro.broker.getvalue()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % end_cash)
    print('PnL Value: %.2f' % (end_cash - start_cash))
    
    cerebro.plot()

In [ ]:
strategy_price_diff = dict.fromkeys(tickers , 0.0)
for index,row in sm._data.iterrows():
    if row["Signal"] == 0:
        continue
     
    strategy_price_diff[index[0]] -= row["Signal"] * row["Close"]
    print(strategy_price_diff)

In [ ]:
start = time.time()
print("hello")
end = time.time()
print(end - start)

In [ ]:
style = mpf.make_mpf_style(base_mpf_style='yahoo', y_on_right=False, edgecolor='Black', facecolor='White')
fig, axs = mpf.plot(sm._data.loc["AAPL"], addplot=[], figscale=1, figratio=(2*8,2*5.75), type='ohlc', volume=True, hlines=[], style=style, title='TEST', returnfig=True)

In [ ]:
runstrat(sm._data.loc["TDOC"])

In [ ]:
sm._data[sm._data['Signal'] != 0]